In this notebook we are classifing csv_result-49_features_downsampled_dataset.csv file using RandomForestClassifier and Stratified K Fold mechanism

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
import numpy as np
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn import metrics
import re
from sklearn import tree
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.tree import export_graphviz
from collections import OrderedDict
!pip install XlsxWriter
import xlsxwriter
from subprocess import call
import csv
import joblib
from sklearn.feature_selection import SelectFromModel

     |████████████████████████████████| 153kB 31.2MB/s 


In [ ]:
df = pd.read_csv("csv_result-49_features_downsampled_dataset.csv", error_bad_lines=False)
df.head()

b'Skipping line 49772: expected 51 fields, saw 53\nSkipping line 49773: expected 51 fields, saw 53\nSkipping line 49774: expected 51 fields, saw 53\nSkipping line 49775: expected 51 fields, saw 53\nSkipping line 49776: expected 51 fields, saw 53\nSkipping line 49777: expected 51 fields, saw 53\nSkipping line 49778: expected 51 fields, saw 53\nSkipping line 49779: expected 51 fields, saw 53\nSkipping line 49780: expected 51 fields, saw 53\nSkipping line 49781: expected 51 fields, saw 53\nSkipping line 49782: expected 51 fields, saw 53\nSkipping line 49783: expected 51 fields, saw 53\nSkipping line 49784: expected 51 fields, saw 53\nSkipping line 49785: expected 51 fields, saw 53\nSkipping line 49786: expected 51 fields, saw 53\nSkipping line 49787: expected 51 fields, saw 53\nSkipping line 49788: expected 51 fields, saw 53\nSkipping line 49789: expected 51 fields, saw 53\nSkipping line 49790: expected 51 fields, saw 53\nSkipping line 49791: expected 51 fields, saw 53\nSkipping line 4979

,id,file_name,corpus_name,sheet_name,sheet_index,table_name,cell_address,cell_type=0,cell_type=2,length,words,leading_spaces,first_char_num,first_char_special,capitalized,all_upper,is_alpha,special_chars,punctuations,contains_colon,words_like_total,words_like_table,in_year_range,is_aggr_formula=1,ref_val_type=0,first_row_num,first_col_num,num_of_neighbors=0,num_of_neighbors=1,num_of_neighbors=2,num_of_neighbors=3,num_of_neighbors=4,h_alignment=0,h_alignment=2,v_alignment=2,indentation,fill_patern=0,is_wraptext,num_of_cells,border_top_type=0,border_top_type=1,border_bottom_type=0,border_left_type=0,border_right_type=0,border_right_type=2,cell_borders=0,font_height,is_font_color_default,is_bold,underline_type=0,label
0,1,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,NaN,A1,f,t,16,3,0,f,f,f,f,f,f,f,f,f,f,f,f,f,0,0,t,f,f,f,f,t,f,t,0,t,f,1,f,f,f,t,t,f,f,12,f,f,t,metadata
1,2,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,NaN,I1,f,t,28,0,0,f,f,f,f,f,f,f,f,f,f,f,f,f,0,8,t,f,f,f,f,f,f,t,0,t,f,1,f,f,f,t,t,f,f,12,f,f,t,metadata
2,3,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,NaN,D8:H8,f,t,28,0,0,f,f,f,f,f,f,f,f,f,f,f,f,f,7,3,t,f,f,f,f,f,t,t,0,t,f,5,f,f,f,f,f,f,f,12,f,t,t,metadata
3,4,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,SHEET1_ANNOTATION_TABLE_B11_J28,C11,f,t,34,4,0,f,f,f,f,f,f,t,f,f,f,f,f,f,10,2,f,t,f,f,f,f,f,t,0,t,f,1,t,f,t,t,t,f,t,10,f,t,f,metadata
4,5,michelle_lokay__26590__IGSUpdate.xls,ENRON,Sheet1,0,SHEET1_ANNOTATION_TABLE_B11_J28,C12,f,t,24,7,0,f,f,f,f,f,f,t,f,f,f,f,f,f,11,2,f,t,f,f,f,f,f,t,0,t,f,1,t,f,t,t,t,f,t,10,f,t,f,metadata


Dropping the columns which are unnecessary


In [ ]:
columns_to_drop = ["id", "file_name", "table_name", "corpus_name", "sheet_name", "sheet_index", "cell_address"]
df = df.drop(columns_to_drop, axis=1)
df.head()

,cell_type=0,cell_type=2,length,words,leading_spaces,first_char_num,first_char_special,capitalized,all_upper,is_alpha,special_chars,punctuations,contains_colon,words_like_total,words_like_table,in_year_range,is_aggr_formula=1,ref_val_type=0,first_row_num,first_col_num,num_of_neighbors=0,num_of_neighbors=1,num_of_neighbors=2,num_of_neighbors=3,num_of_neighbors=4,h_alignment=0,h_alignment=2,v_alignment=2,indentation,fill_patern=0,is_wraptext,num_of_cells,border_top_type=0,border_top_type=1,border_bottom_type=0,border_left_type=0,border_right_type=0,border_right_type=2,cell_borders=0,font_height,is_font_color_default,is_bold,underline_type=0,label
0,f,t,16,3,0,f,f,f,f,f,f,f,f,f,f,f,f,f,0,0,t,f,f,f,f,t,f,t,0,t,f,1,f,f,f,t,t,f,f,12,f,f,t,metadata
1,f,t,28,0,0,f,f,f,f,f,f,f,f,f,f,f,f,f,0,8,t,f,f,f,f,f,f,t,0,t,f,1,f,f,f,t,t,f,f,12,f,f,t,metadata
2,f,t,28,0,0,f,f,f,f,f,f,f,f,f,f,f,f,f,7,3,t,f,f,f,f,f,t,t,0,t,f,5,f,f,f,f,f,f,f,12,f,t,t,metadata
3,f,t,34,4,0,f,f,f,f,f,f,t,f,f,f,f,f,f,10,2,f,t,f,f,f,f,f,t,0,t,f,1,t,f,t,t,t,f,t,10,f,t,f,metadata
4,f,t,24,7,0,f,f,f,f,f,f,t,f,f,f,f,f,f,11,2,f,t,f,f,f,f,f,t,0,t,f,1,t,f,t,t,t,f,t,10,f,t,f,metadata


Converting categorical columns to ordinal

In [ ]:
 categorical_cols = ['cell_type=0', 'cell_type=2', 'first_char_num', 'first_char_special', 'capitalized', 
    'all_upper', 'is_alpha', 'special_chars', 'punctuations', 'contains_colon', 'words_like_total', 
    'words_like_table', 'in_year_range', 'is_aggr_formula=1', 'ref_val_type=0', 'num_of_neighbors=0', 
    'num_of_neighbors=1', 'num_of_neighbors=2', 'num_of_neighbors=3', 'num_of_neighbors=4', 'h_alignment=0', 
    'h_alignment=2', 'v_alignment=2', 'fill_patern=0', 'is_wraptext', 'border_top_type=0', 
    'border_top_type=1', 'border_bottom_type=0', 'border_left_type=0', 'border_right_type=0', 
    'border_right_type=2', 'cell_borders=0', 'is_font_color_default', 'is_bold', 'underline_type=0']
for colName in categorical_cols:
  df[colName] = df[colName].astype('category')
cat_columns = df.select_dtypes(['category']).columns
df[cat_columns] = df[cat_columns].apply(lambda x: x.cat.codes)
df.head()

,cell_type=0,cell_type=2,length,words,leading_spaces,first_char_num,first_char_special,capitalized,all_upper,is_alpha,special_chars,punctuations,contains_colon,words_like_total,words_like_table,in_year_range,is_aggr_formula=1,ref_val_type=0,first_row_num,first_col_num,num_of_neighbors=0,num_of_neighbors=1,num_of_neighbors=2,num_of_neighbors=3,num_of_neighbors=4,h_alignment=0,h_alignment=2,v_alignment=2,indentation,fill_patern=0,is_wraptext,num_of_cells,border_top_type=0,border_top_type=1,border_bottom_type=0,border_left_type=0,border_right_type=0,border_right_type=2,cell_borders=0,font_height,is_font_color_default,is_bold,underline_type=0,label
0,0,1,16,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0,1,0,0,0,1,1,0,0,12,0,0,1,metadata
1,0,1,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,1,0,0,12,0,0,1,metadata
2,0,1,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,3,1,0,0,0,0,0,1,1,0,1,0,5,0,0,0,0,0,0,0,12,0,1,1,metadata
3,0,1,34,4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,10,2,0,1,0,0,0,0,0,1,0,1,0,1,1,0,1,1,1,0,1,10,0,1,0,metadata
4,0,1,24,7,0,0,0,0,0,0,0,1,0,0,0,0,0,0,11,2,0,1,0,0,0,0,0,1,0,1,0,1,1,0,1,1,1,0,1,10,0,1,0,metadata


Initializing Random Forest classifier

In [ ]:
clf = RandomForestClassifier(bootstrap=True,
 criterion= 'entropy',
 max_depth= 15,
 max_features= 'auto',
 n_estimators= 500)

Fitting a full model and saving it

In [ ]:
df_full = df
y = df_full['label'].values
X = df_full.drop(['label'], axis=1)
X = X.to_numpy()

clf.fit(X, y)

filename = 'finalized_model_full.sav'
joblib.dump(clf, filename)

['finalized_model_full.sav']

spliting the rows 80-20% and fiiting the model

In [ ]:
df_full = df
y = df_full['label'].values
X = df_full.drop(['label'], axis=1)
X = X.to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Predicting the test data on the previous fit model
ALso generating a confusion matrix

In [ ]:
 y_full_predict = []
 j = 0
 for row in X_test:
    prediction = clf.predict([row])
    y_full_predict.append(prediction)
    j+=1

confusion_mtx = metrics.confusion_matrix(y_test, y_full_predict, labels = ['data', 'header', 'attributes', 'metadata', 'derived'])
print(confusion_mtx)
print(metrics.classification_report(y_test, y_full_predict, labels=['data', 'header', 'attributes', 'metadata', 'derived']))

[[6578   27   23    9    6]
 [  63 1861   11    2    1]
 [  28    5  794    2    0]
 [  32   22    6  582    0]
 [  60    0    0    0  460]]
              precision    recall  f1-score   support

        data       0.97      0.99      0.98      6643
      header       0.97      0.96      0.97      1938
  attributes       0.95      0.96      0.95       829
    metadata       0.98      0.91      0.94       642
     derived       0.99      0.88      0.93       520

    accuracy                           0.97     10572
   macro avg       0.97      0.94      0.96     10572
weighted avg       0.97      0.97      0.97     10572



Saving the 80-20 model 

In [ ]:
filename = 'finalized_model_full_80.sav'
joblib.dump(clf, filename)

['finalized_model_full_80.sav']

In [ ]:
y_pred_five_arr = []
y_test_five_arr = []
cross_val = StratifiedKFold(n_splits=10)
def fiveClassClassifier(df):
  df_three = df
  Y = df_three['label'].values
  X = df_three.drop(['label'], axis=1)
  header = X.columns.to_numpy()
  i = 0
  for train_index, test_index in cross_val.split(X, Y):
    X_train = X.to_numpy()[train_index]
    y_train = Y[train_index]
    X_test = X.to_numpy()[test_index]
    y_test = Y[test_index]
    clf.fit(X_train, y_train)
    filename = 'finalized_model_five_'+str(i)+'.sav'
    joblib.dump(clf, filename)
    # y_test = y_test.to_numpy()

    # fp = open("op_rfc_strat"+str(i)+".csv","w+") 
    # writer = csv.writer(fp, quoting=csv.QUOTE_NONNUMERIC)
    
    # Open an Excel workbook
    # Set up a format
    # Create a sheet
    # workbook = xlsxwriter.Workbook('op_rfc_strat'+str(i)+'.xlsx')
    # book_format = workbook.add_format(properties={'bold': True, 'font_color': 'red'})
    # worksheet = workbook.add_worksheet('sheet1')

    y_predicted = []
    # X_test = X_test.to_numpy()
    # j = 0
    # header = np.append(header, "actual")
    # header = np.append(header, "prediction")
    # lst = header.tolist()
    # writer.writerow(lst)
    # for col_num, cell_data in enumerate(lst):
    #   worksheet.write(0, col_num, cell_data)
    
    for row in X_test:
      prediction = clf.predict([row])
      y_predicted.append(prediction)
      # arr = row
      # arr = np.append(arr, y_test[j])
      # arr = np.append(arr, prediction)
      # lst = arr.tolist()
      # if prediction != y_test[j]:
      #   writer.writerow(lst)
      #   for col_num, cell_data in enumerate(lst):
      #     worksheet.write(j+1, col_num, cell_data, book_format)
      #   # file1.writerows(row + " actual:" + y_test[j] + " predicted:" + prediction)
      #   # print(row , " actual:" , y_test[j] , " predicted:" , prediction)
      # else:
      #   for col_num, cell_data in enumerate(lst):
      #     worksheet.write(j+1, col_num, cell_data)
      # j+=1
      
    # y_test = y_test.to_numpy()
    print(y_test)
    print(y_predicted)
    # fp.close()
    # workbook.close()

    y_pred_five_arr.append(y_predicted)
    y_test_five_arr.append(y_test)
    i += 1

In [ ]:
fiveClassClassifier(df)

['metadata' 'metadata' 'metadata' ... 'metadata' 'metadata' 'metadata']
[array(['metadata'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['header'], dtype=object), array(['metadata'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['metadata'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['data'], dtype=object), array(['attributes'], dtype=object), array(['data'], dtype=object), array(['data'], dty

In [ ]:
def printMetrics(y_pred_arr, y_test_arr, target_names):
  combined = [[0 for j in range(len(target_names))] for i in range (len(target_names))]
  print(combined)

  for i in range(10):
    count = 0
    y_predicted = y_pred_arr[i]
    y_test = y_test_arr[i]
    for j in range(len(y_predicted)):
      pred = y_predicted[j][0]
      if(pred != y_test[j]):
        count += 1

    print("Fold:", i+1)

    count = Counter(y_test)
    for target in target_names:
      print(target, ":", count[target])
    confusion_mtx = metrics.confusion_matrix(y_test, y_predicted, labels = target_names)
    combined = [[combined[i][j] + confusion_mtx[i][j]  for j in range(len(target_names))] for i in range(len(target_names))]
    print(confusion_mtx)
    print(metrics.classification_report(y_test, y_predicted, labels=target_names))
    print("------------")


  for i in range(len(combined)):
    print(combined[i])
  recall = np.diag(combined) / np.sum(combined, axis = 1)
  precision = np.diag(combined) / np.sum(combined, axis = 0)
  f1Score = []
  for i in range(5):
    f1Score.append((2*recall[i]*precision[i])/(recall[i] + precision[i]))
    
  print("precision", "recall", "f1Score")
  for i in range(len(recall)) :
      print(target_names[i], ":", precision[i], recall[i], f1Score[i])


In [ ]:
printMetrics(y_pred_five_arr, y_test_five_arr,  ['data', 'header', 'attributes', 'metadata', 'derived'])

[[0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0]]
Fold: 1
data : 3281
header : 978
attributes : 422
metadata : 336
derived : 269
[[3066  101   38   22   54]
 [ 153  704  102   19    0]
 [ 166   55  183   18    0]
 [  99  105    9  121    2]
 [  97    0    0    0  172]]
              precision    recall  f1-score   support

        data       0.86      0.93      0.89      3281
      header       0.73      0.72      0.72       978
  attributes       0.55      0.43      0.49       422
    metadata       0.67      0.36      0.47       336
     derived       0.75      0.64      0.69       269

    accuracy                           0.80      5286
   macro avg       0.71      0.62      0.65      5286
weighted avg       0.79      0.80      0.79      5286

------------
Fold: 2
data : 3281
header : 978
attributes : 422
metadata : 336
derived : 269
[[3174   63   25    5   14]
 [ 356  605    2   15    0]
 [ 154    0  268    0    0]
 [  44   50   47  195    0]
 

In [ ]:
y_pred_three_arr = []
y_test_three_arr = []
cross_val = StratifiedKFold(n_splits=10)
def threeClassClassifier(df):
  df_three = df
  df_three['label'] = df_three['label'].replace(['attributes'], 'header')
  df_three['label'] = df_three['label'].replace(['derived'], 'data')
  Y = df_three['label'].values
  X = df_three.drop(['label'], axis=1)
  header = X.columns.to_numpy()
  i = 0
  for train_index, test_index in cross_val.split(X, Y):
    X_train = X.to_numpy()[train_index]
    y_train = Y[train_index]
    X_test = X.to_numpy()[test_index]
    y_test = Y[test_index]
    clf.fit(X_train, y_train)
    filename = 'finalized_model_three_'+str(i)+'.sav'
    joblib.dump(clf, filename)
    # y_test = y_test.to_numpy()

    fp = open("op_rfc_strat"+str(i)+".csv","w+") 
    writer = csv.writer(fp, quoting=csv.QUOTE_NONNUMERIC)
    
    # Open an Excel workbook
    # Set up a format
    # Create a sheet
    workbook = xlsxwriter.Workbook('op_rfc_strat'+str(i)+'.xlsx')
    book_format = workbook.add_format(properties={'bold': True, 'font_color': 'red'})
    worksheet = workbook.add_worksheet('sheet1')

    y_predicted = []
    # X_test = X_test.to_numpy()
    j = 0
    header = np.append(header, "actual")
    header = np.append(header, "prediction")
    lst = header.tolist()
    writer.writerow(lst)
    for col_num, cell_data in enumerate(lst):
      worksheet.write(0, col_num, cell_data)
    
    for row in X_test:
      prediction = clf.predict([row])
      y_predicted.append(prediction)
      arr = row
      arr = np.append(arr, y_test[j])
      arr = np.append(arr, prediction)
      lst = arr.tolist()
      if prediction != y_test[j]:
        writer.writerow(lst)
        for col_num, cell_data in enumerate(lst):
          worksheet.write(j+1, col_num, cell_data, book_format)
        # file1.writerows(row + " actual:" + y_test[j] + " predicted:" + prediction)
        # print(row , " actual:" , y_test[j] , " predicted:" , prediction)
      else:
        for col_num, cell_data in enumerate(lst):
          worksheet.write(j+1, col_num, cell_data)
      j+=1
      
    print(y_predicted)
    # y_test = y_test.to_numpy()
    print(y_test)
    fp.close()
    workbook.close()

    y_pred_three_arr.append(y_predicted)
    y_test_three_arr.append(y_test)

    # y_pred_three_arr.append(y_predicted)
    # y_test_three_arr.append(y_test)
    i += 1

In [ ]:
# threeClassClassifier(df)

In [ ]:
# printMetrics(y_pred_three_arr, y_test_three_arr,  ['data', 'header', 'metadata'])

In [ ]:
from sklearn.model_selection import GridSearchCV
rfc=RandomForestClassifier(random_state=42)
param_grid = { 
    'n_estimators': [200, 300, 400, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [None, 5, 10],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=10)


def fiveClassClassifierGridSearch(df):
  Y = df['label'].values
  X = df.drop(['label'], axis=1)
  header = X.columns.to_numpy()
  CV_rfc.fit(X, Y)
  CV_rfc.best_params_

# fiveClassClassifierGridSearch(df)
# CV_rfc.best_params_